<div style="text-align: center; font-size: 16px;">
    <strong>Course:</strong> Machine Learning Operations |
    <strong>Lecturer:</strong> Prof. Dr. Klotz |
    <strong>Date:</strong> 17.05.2025 |
    <strong>Name:</strong> Sofie Pischl
</div>

# <center>Data Collection </center>

Konzept & Inhalt:

Daten von den größten Social media Apps sollen abgegriffen werden. besonderer Fokus auf texten.

1. Reddit
2. Instagram

In [10]:
import os
import praw
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv

# Load from .env file
load_dotenv()


True

# 1. Reddit

In [ ]:

# 🔑 Authentifizierung (trage deine echten Keys ein)
reddit = praw.Reddit(
    client_id=(os.getenv("REDDIT_ID")),
    client_secret=(os.getenv("REDDIT_SECRET")),
    user_agent=(os.getenv("USER_AGENT"))
)

# 🔍 Subreddits und Query definieren
subreddits = ["books", "writing", "technology", "future"]
post_limit = 100

all_posts = []

for sub in subreddits:
    subreddit = reddit.subreddit(sub)
    for post in subreddit.hot(limit=post_limit):
        all_posts.append({
            "subreddit": sub,
            "title": post.title,
            "text": post.selftext,
            "score": post.score,
            "comments": post.num_comments,
            "created": datetime.fromtimestamp(post.created),
            "url": post.url
        })

# 📄 In DataFrame umwandeln & speichern
df = pd.DataFrame(all_posts)
df.to_csv("../data/raw/reddit_data.csv", index=False)
df.head()


,subreddit,title,text,score,comments,created,url
0,books,"Weekly FAQ Thread March 09, 2025: What are the...",Hello readers and welcome to our Weekly FAQ th...,54,96,2025-03-09 12:00:31,https://www.reddit.com/r/books/comments/1j75ax...
1,books,"Weekly FAQ Thread April 06, 2025: What are you...",Hello readers and welcome to our Weekly FAQ th...,13,17,2025-04-06 13:00:31,https://www.reddit.com/r/books/comments/1jsrko...
2,books,Romance author Ali Hazelwood cancels UK tours ...,,6476,423,2025-04-05 18:55:34,https://www.usatoday.com/story/entertainment/b...
3,books,I want to buy new books at the book fair and b...,"In Japanese, there's a word for people who buy...",213,357,2025-04-06 12:48:15,https://www.reddit.com/r/books/comments/1jsre8...
4,books,'Oliver and Amanda Pig' series author Jean Van...,,48,1,2025-04-06 12:09:48,https://www.npr.org/2025/04/05/nx-s1-5353478/o...


---

# 2. Instagram

was nicht funktioniert hat: instaloader, playwright

step 1: log in & extract urls from for you page

step 2: Look up urls without beeing signed in and extract captions & content

setp 1: 

In [ ]:
import time
import re
import os
import pandas as pd
from selenium import webdriver
from dotenv import load_dotenv
load_dotenv()
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

In [18]:
# log in data
USERNAME =  os.getenv("INSTA_USERNAME")
PASSWORD =  os.getenv("INSTA_PASSWORD")

In [ ]:
# Start browser and login
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
wait = WebDriverWait(driver, 15)

driver.get("https://www.instagram.com/accounts/login/")
time.sleep(3)

# Accept cookies (if shown)
try:
    decline_button = wait.until(EC.element_to_be_clickable(
        (By.XPATH, '//button[contains(text(), "Nur essentielle Cookies erlauben") or contains(text(), "Decline optional cookies")]')))
    decline_button.click()
except:
    pass

# Fill in login form
wait.until(EC.presence_of_element_located((By.NAME, 'username')))
driver.find_element(By.NAME, 'username').send_keys(USERNAME)
driver.find_element(By.NAME, 'password').send_keys(PASSWORD)
driver.find_element(By.NAME, 'password').send_keys(Keys.RETURN)

time.sleep(5)

# Go to Explore page
driver.get("https://www.instagram.com/explore/")
time.sleep(5)

# Scroll and collect post URLs
post_urls = set()
scrolls = 0
while len(post_urls) < 20 and scrolls < 10:
    elements = driver.find_elements(By.XPATH, "//a[contains(@href, '/p/')]")
    for elem in elements:
        href = elem.get_attribute("href")
        if href and href.startswith("https://www.instagram.com/p/"):
            post_urls.add(href)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    scrolls += 1

# Save URLs to CSV
df = pd.DataFrame({"Post URL": list(post_urls)})
df.to_csv("../raw/explore_results.csv", index=False)
print(f"✅ Saved {len(post_urls)} URLs to 'explore_results.csv'.")

driver.quit()

✅ Saved 20 URLs to 'explore_results.csv'.


In [ ]:

# === CONFIGURATION ===
csv_path = "../data/raw/explore_results.csv"

# === READ CSV AND PREPARE DATAFRAME ===
# Ensure columns are interpreted as strings
df = pd.read_csv(csv_path, dtype={
    "timestamp": "string",
    "datum": "string",
    "inhalt": "string",
    "username": "string",
    "caption": "string",
    "likes": "string"
})

# Ensure necessary columns exist
required_columns = ["timestamp", "datum", "inhalt", "username", "caption", "likes"]
for col in required_columns:
    if col not in df.columns:
        df[col] = pd.Series(dtype="string")

# === SETUP SELENIUM BROWSER ===
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 15)

# === DISMISS POPUPS ===
def dismiss_popups():
    # Cookie-Banner
    cookie_buttons = [
        '//button[text()="Nur essentielle Cookies erlauben"]',
        '//button[text()="Decline optional cookies"]',
    ]

    # Login-Overlay schließen
    login_close_xpaths = [
        '//div[@role="dialog"]//div[@aria-label="Schließen"]',
        '//div[@role="dialog"]//div[@aria-label="Close"]',
        '//button[@aria-label="Schließen"]',
        '//button[@aria-label="Close"]',
    ]

    for xpath in cookie_buttons + login_close_xpaths:
        try:
            btn = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, xpath)))
            btn.click()
            time.sleep(1)
        except:
            continue

# === MAIN LOOP: PROCESS EACH POST ===
for i, row in df.iterrows():
    url = row['Post URL']
    if pd.notna(row.get("timestamp")):
        continue  # Skip already processed rows

    try:
        driver.get(url)
        time.sleep(1)
        dismiss_popups()

        # Check if article appears – if not: skip (possibly 18+ or deleted)
        try:
            wait.until(EC.presence_of_element_located((By.TAG_NAME, "article")))
        except:
            print(f" Beitrag nicht verfügbar oder 18+: {url}")
            continue


        # Username
        try:
            username_elem = driver.find_element(By.XPATH, "//a[contains(@href, '/') and contains(@class, 'notranslate')]//span").text.strip()
            username = username_elem.text.strip()
        except:
            username = ""
        
        # Caption (try newer & older structure)
        try:
            all_spans = driver.find_elements(By.XPATH, "//div[contains(@class, '_a9zs')]/span").text.strip()
            caption = ""
            for span in all_spans:
                text = span.text.strip()
                if text and username not in text and not re.search(r"Gefällt|Kommentieren|Speichern", text):
                    caption = text
                    break
        except:
            caption = ""

        # Alt-Text (Image Description)
        try:
            img = driver.find_element(By.XPATH, "//article//img")
            image_alt = img.get_attribute("alt").strip()
        except:
            image_alt = ""

        # Datum
        try:
            date_elem = driver.find_element(By.XPATH, "//time")
            post_date = date_elem.get_attribute("datetime")[:10]
        except:
            post_date = ""

        # Likes (only if visible)
        try:
            likes_elem = driver.find_element(By.XPATH, "//section//span[contains(text(),'Gefällt')]").text.strip()
            likes_text = likes_elem.text
            likes_number = re.findall(r"\d[\d\.\,]*", likes_text)
            likes = int(likes_number[0].replace(".", "").replace(",", "")) if likes_number else ""
        except:
            likes = ""

        # Save to DataFrame
        df.at[i, "timestamp"] = datetime.now().isoformat()
        df.at[i, "datum"] = post_date
        df.at[i, "inhalt"] = image_alt
        df.at[i, "username"] = username
        df.at[i, "caption"] = caption
        df.at[i, "likes"] = str(likes)

        print(f"✅ Saved: {username} | {caption[:30]}...")

    except Exception as e:
        print(f"❌ Error with {url}: {e}")
        continue

# === FINAL SAVE ===
df.to_csv(csv_path, index=False, encoding='utf-8')
driver.quit()
print("✅ All data saved to explore_results.csv")


 Beitrag nicht verfügbar oder 18+: https://www.instagram.com/p/DIKaU11Rwj0/


In [5]:
df.head()


,Post URL,timestamp,datum,inhalt,username,caption,likes
0,https://www.instagram.com/p/DIKaU11Rwj0/,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,https://www.instagram.com/p/DHW6TrpR9UE/,2025-04-14T12:47:05.156202,2025-03-18,"Photo by BernieGirl on March 18, 2025. Ist mög...",<NA>,<NA>,<NA>
2,https://www.instagram.com/p/DHOLtvLtJKL/,2025-04-14T12:47:26.154501,2025-03-15,Photo by Die Welt hinter der Leinwand on March...,<NA>,<NA>,<NA>
3,https://www.instagram.com/p/DH-iv39t-RD/,2025-04-14T12:47:55.882011,2025-04-03,aiwonderlab.eu's profile picture,<NA>,<NA>,<NA>
4,https://www.instagram.com/p/DIJwgPKpxTN/,2025-04-14T12:48:16.988772,2025-04-07,sonya_styless's profile picture,<NA>,<NA>,<NA>


# 1. TWITTER

In [4]:
import os

In [9]:
!pip install python-dotenv

In [ ]:
import requests
import pandas as pd
from datetime import datetime

# Your bearer token from Twitter Developer Portal
BEARER_TOKEN = os.getenv("X_BEARER_TOKEN")

# Twitter API endpoint for recent tweets
search_url = "https://api.twitter.com/2/tweets/search/recent"

# Search parameters – open topic
query_params = {
    'query': 'lang:de -is:retweet',  # No keyword, just German tweets
    'max_results': 50,  # Max per request (10–100)
    'tweet.fields': 'created_at,public_metrics,text,author_id',
    'expansions': 'author_id',
    'user.fields': 'username,name'
}

# Set headers
headers = {
    "Authorization": f"Bearer {BEARER_TOKEN}"
}

# Send request
response = requests.get(search_url, headers=headers, params=query_params)

# Check response
if response.status_code != 200:
    raise Exception(f"Request failed: {response.status_code}\n{response.text}")

data = response.json()

# Extract data
tweets = data.get("data", [])
users = {u["id"]: u for u in data.get("includes", {}).get("users", [])}

# Prepare data rows
results = []
for tweet in tweets:
    user = users.get(tweet["author_id"], {})
    metrics = tweet.get("public_metrics", {})
    results.append({
        "url": f"https://twitter.com/{user.get('username')}/status/{tweet['id']}",
        "timestamp": datetime.now().isoformat(),
        "datum": tweet.get("created_at", ""),
        "username": user.get("username", ""),
        "name": user.get("name", ""),
        "caption": tweet.get("text", ""),
        "likes": metrics.get("like_count", 0),
        "retweets": metrics.get("retweet_count", 0),
        "replies": metrics.get("reply_count", 0)
    })

# Save to CSV
df = pd.DataFrame(results)
df.sort_values(by="likes", ascending=False, inplace=True)  # Sort by popularity
df.to_csv("../raw/twitter_api_top_tweets.csv", index=False, encoding="utf-8")

print(f"Saved {len(df)} tweets to twitter_api_top_tweets.csv")


Exception: Request failed: 429
{"title":"Too Many Requests","detail":"Too Many Requests","type":"about:blank","status":429}